In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
!pip freeze > requirements.txt

In [ ]:
data_dir = 'mod_fol'
train_dir = data_dir + '/train'
val_dir = data_dir + '/validation'
test_dir = data_dir + '/test'

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_set = ImageFolder(train_dir, transform=train_transform)
val_set = ImageFolder(val_dir, transform=val_transform)
test_set = ImageFolder(test_dir, transform=test_transform)


In [ ]:
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)


In [ ]:
model = models.mobilenet_v2(pretrained=True)
num_classes = len(train_set.classes)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)  # Modify the last layer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
num_epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 10 == 9:  # Print loss every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/10:.4f}")
            running_loss = 0.0

Epoch [1/1], Batch [10/98], Loss: 0.5195
Epoch [1/1], Batch [20/98], Loss: 0.5241
Epoch [1/1], Batch [30/98], Loss: 0.4932
Epoch [1/1], Batch [40/98], Loss: 0.6032
Epoch [1/1], Batch [50/98], Loss: 0.5038
Epoch [1/1], Batch [60/98], Loss: 0.5326
Epoch [1/1], Batch [70/98], Loss: 0.5472
Epoch [1/1], Batch [80/98], Loss: 0.5131
Epoch [1/1], Batch [90/98], Loss: 0.4918


In [ ]:
model.eval()
val_accuracy = 0.0
val_total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total += labels.size(0)
        val_accuracy += (predicted == labels).sum().item()

print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Validation Accuracy: {100 * val_accuracy / val_total:.2f}%")

Epoch 1/1, Loss: 4.1541, Validation Accuracy: 92.31%


In [ ]:
torch.save(model.state_dict(), 'modelforclass.pth')

In [ ]:
from PIL import Image

# Step 14: Load the trained model
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model.load_state_dict(torch.load('modelforclass.pth'))
model.to(device)
model.eval()

# Step 15: Preprocess the input image
input_image = Image.open('beetrootone.jpg')
input_image = test_transform(input_image)
input_image = input_image.unsqueeze(0)  # Add a batch dimension
input_image = input_image.to(device)  # Move the input image to the same device as the model

# Step 16: Classify the input image
with torch.no_grad():
    output = model(input_image)

# Step 17: Get the predicted class label
_, predicted_idx = torch.max(output, 1)
predicted_label = train_set.classes[predicted_idx.item()]

# Step 18: Print the predicted label
print("Predicted Label:", predicted_label)


Predicted Label: beetroot
